# Обучение модели YOLOv5






## ЗАДАНИЕ

Обучите модель **yolov5**,  используя готовый `train.py` файл репозитория https://github.com/ultralytics/yolov5.git.

Для данной модели изображения и классы объектов должны находиться в папках images и labels, соответсвенно.

Протестируйте модель на валидациооной выборке и выведите на экран полученные изображения с bbox и классами объектов.

Сделайте сравнение текущей модели с Faster RCNN по скорости и качетсву.

### Импорт репозитория

In [2]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...


### Установка необходимых пакетов

In [ ]:
!pip install -r yolov5/requirements.txt

### Импрот необходимых модулей

In [3]:
import torch
from IPython.display import Image  # for displaying images
import os 
import random
import shutil
from sklearn.model_selection import train_test_split
import xml.etree.ElementTree as ET
from xml.dom import minidom
from tqdm import tqdm
from PIL import Image, ImageDraw
import numpy as np
import matplotlib.pyplot as plt

import urllib.request
import tarfile

### Загрузка данных

In [5]:
# Загрузка датасета
data_dir = "../../0000_data/"
if not os.path.exists(f"{data_dir}VOCdevkit"):
    if not os.path.exists(f"{data_dir}VOC.tar"):
        urllib.request.urlretrieve(
            "https://pjreddie.com/media/files/VOCtrainval_11-May-2012.tar", f"{data_dir}VOC.tar"
        )
    with tarfile.open(f"{data_dir}VOC.tar") as tar:
        tar.extractall()

### Конвертация аннотаций к формату YOLO v5


### Формат PASCAL VOC 
хранит свои аннотации в XML-файлах. Давайте посмотрим на один такой файл.

In [8]:
!type ..\..\0000_data\VOCdevkit\VOC2012\Annotations\2007_000027.xml 

<annotation>
	<folder>VOC2012</folder>
	<filename>2007_000027.jpg</filename>
	<source>
		<database>The VOC2007 Database</database>
		<annotation>PASCAL VOC2007</annotation>
		<image>flickr</image>
	</source>
	<size>
		<width>486</width>
		<height>500</height>
		<depth>3</depth>
	</size>
	<segmented>0</segmented>
	<object>
		<name>person</name>
		<pose>Unspecified</pose>
		<truncated>0</truncated>
		<difficult>0</difficult>
		<bndbox>
			<xmin>174</xmin>
			<ymin>101</ymin>
			<xmax>349</xmax>
			<ymax>351</ymax>
		</bndbox>
		<part>
			<name>head</name>
			<bndbox>
				<xmin>169</xmin>
				<ymin>104</ymin>
				<xmax>209</xmax>
				<ymax>146</ymax>
			</bndbox>
		</part>
		<part>
			<name>hand</name>
			<bndbox>
				<xmin>278</xmin>
				<ymin>210</ymin>
				<xmax>297</xmax>
				<ymax>233</ymax>
			</bndbox>
		</part>
		<part>
			<name>foot</name>
			<bndbox>
				<xmin>273</xmin>
				<ymin>333</ymin>
				<xmax>297</xmax>
				<ymax>354</ymax>
			</bndbox>
		</part>
		<part>
			<name>foot<

Данная аннотация относится к файлу 2007_000027.jpg размером 486х500х3. Файл содерит один тэг object, описывающий класс объекта и координаты ограничивающих рамок. Класс определяется тегом name. Рамка тегами - xmin, ymin, xmax, ymax. Соответственно верхний левый угол рамки (xmin, ymin) и нижний правый (xmax, ymax). Четыре тега part описывают рамки частей объекта.

### Формат YOLO v5 
подразумевает аннотацию для каждого изображения в виде файла .txt (имя файл совпадает с именем изображения), где каждая строка текстового файла описывает ограничивающую рамку. Рассмотрим следующее изображение.

![picture](https://drive.google.com/uc?export=view&id=1yUJt5XTvYo4MbWxKAxhPdCk-yCnQ1AEz)




Файл аннотаций zidane.txt для приведенного выше изображения выглядит следующим образом:

```
27 0.786719 0.502778 0.0328125 0.161111 0.287125
0 0.471875 0.629167 0.765625 0.713889 0.593856
0 0.741406 0.522222 0.310937 0.925 0.877349
```
Описано три объекта. Спецификация для каждой строки выглядит следующим образом:

* Одна строка на объект
* Каждая строка имеет формат class x_center y_center width height.
* Координаты нормализованы по относительно размера изображения
* Индексация классов начинается с нуля

Структура каталогов набора данных формата YOLOv5
```
base_dir
├── images           # изображения
│   ├── test
│   ├── train
│   └── validation
└── labels           # аннотации
    ├── test
    ├── train
    └── validation
```    
